# The Traveling Salesman Problem (TSP)

This is perhaps the most notorious problem in Operations Research because it is so easy to explain, and so tempting to try and solve. A salesman must visit each of $n$ cities exactly once and then return to his starting point. The time taken to travel from city $i$ to city $j$ is $c_{ij}$. Find the order in which he should make his tour as short as possible.

This problem arises in a multitude of forms: a truck driver has a list of clients he must visit on a given day, or a machine must place modules on printed circuit boards, or a stacker crane must pick up and deposit crates. Now we formulate it as a Binary Integer Program (BIP).

## Decision Variables

$$
x_{ij} =
\begin{cases}
1 & \text{if the salesman goes directly from city } i \text{ to city } j \\
0 & \text{otherwise}
\end{cases}
\quad \text{for } i,j = 1,\ldots,n, i \neq j.
$$

## Constraints

1. The salesman leaves each city exactly once:

$$
\sum_{\substack{j=1 \\ j \neq i}}^{n} x_{ij} = 1 \quad \text{for } i = 1,\ldots,n.
$$

2. The salesman arrives at each city exactly once:

$$
\sum_{\substack{i=1 \\ i \neq j}}^{n} x_{ij} = 1 \quad \text{for } j = 1,\ldots,n.
$$

3. Subtour elimination constraints (to ensure a single complete tour):

$$
\sum_{i \in S} \sum_{j \notin S} x_{ij} \geq 1 \quad \text{for all } S \subset N, S \neq \emptyset.
$$

Alternatively, using the Dantzig-Fulkerson-Johnson formulation:

$$
\sum_{i \in S} \sum_{j \in S} x_{ij} \leq |S| - 1 \quad \text{for all } S \subset N, 2 \leq |S| \leq n-1.
$$

4. Binary constraints:

$$
x_{ij} \in \{0, 1\} \quad \text{for } i,j = 1,\ldots,n, i \neq j.
$$

## Objective Function

Minimize the total travel time:

$$
\min \sum_{i=1}^{n} \sum_{\substack{j=1 \\ j \neq i}}^{n} c_{ij} x_{ij}.
$$

## Complete BIP Formulation

$$
\begin{aligned}
& \min \sum_{i=1}^{n} \sum_{\substack{j=1 \\ j \neq i}}^{n} c_{ij} x_{ij} \\
& \text{subject to:} \\
& \quad \sum_{\substack{j=1 \\ j \neq i}}^{n} x_{ij} = 1, \quad i = 1,\ldots,n \\
& \quad \sum_{\substack{i=1 \\ i \neq j}}^{n} x_{ij} = 1, \quad j = 1,\ldots,n \\
& \quad \sum_{i \in S} \sum_{j \in S} x_{ij} \leq |S| - 1, \quad \text{for all } S \subset N, 2 \leq |S| \leq n-1 \\
& \quad x_{ij} \in \{0, 1\}, \quad i,j = 1,\ldots,n, i \neq j
\end{aligned}
$$


In [1]:
!pip install gurobipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 88.2 MB/s eta 0:00:00


In [2]:
import gurobipy as gp
from gurobipy import GRB
import itertools
import random
import math

In [3]:
cities = ["A", "B", "C", "D"]
distances = {
    ("A", "B"): 10, ("A", "C"): 15, ("A", "D"): 20,
    ("B", "C"): 35, ("B", "D"): 25,
    ("C", "D"): 30
}
# Symmetric completion
for (i, j), d in list(distances.items()):
  distances[(j, i)] = d

m = gp.Model("TSP")
x = m.addVars(distances.keys(), vtype=GRB.BINARY, name="x")
m.setObjective(gp.quicksum(distances[i, j] * x[i, j] for (i, j) in distances), GRB.MINIMIZE) # Objective
"""Constraints: each city has exactly one in and one out"""
for k in cities:
  m.addConstr(gp.quicksum(x[i, j] for (i, j) in distances if i == k) == 1)
  m.addConstr(gp.quicksum(x[i, j] for (i, j) in distances if j == k) == 1)
m.optimize()

print("Tour edges:")
edges = [(i, j) for (i, j) in distances if x[i, j].X > 0.5]
print(edges)
print("Total cost:", m.objVal, "\n")

Restricted license - for non-production use only - expires 2026-11-23
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 8 rows, 12 columns and 24 nonzeros
Model fingerprint: 0xc6b885b3
Variable types: 0 continuous, 12 integer (12 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+01, 4e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 95.0000000
Presolve time: 0.00s
Presolved: 8 rows, 12 columns, 24 nonzeros
Variable types: 0 continuous, 12 integer (12 binary)

Root relaxation: objective 8.000000e+01, 8 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Nod

In [4]:
cities = ["W", "X", "Y", "Z"]
distances = {
    "W": {"X": 12, "Y": 10, "Z": 19},
    "X": {"W": 12, "Y": 3, "Z": 7},
    "Y": {"W": 10, "X": 3, "Z": 6},
    "Z": {"W": 19, "X": 7, "Y": 6},
}

pairs = [(i, j) for i in cities for j in cities if i != j]
m = gp.Model("TSP_dict")
x = m.addVars(pairs, vtype=GRB.BINARY, name="x")
m.setObjective(gp.quicksum(distances[i][j] * x[i, j] for (i, j) in pairs), GRB.MINIMIZE)
for k in cities:
  m.addConstr(gp.quicksum(x[i, j] for (i, j) in pairs if i == k) == 1)
  m.addConstr(gp.quicksum(x[i, j] for (i, j) in pairs if j == k) == 1)
m.optimize()

print("Tour edges:")
edges = [(i, j) for (i, j) in pairs if x[i, j].X > 0.5]
print(edges)
print("Total cost:", m.objVal, "\n")

Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 8 rows, 12 columns and 24 nonzeros
Model fingerprint: 0x607b794c
Variable types: 0 continuous, 12 integer (12 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e+00, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 36.0000000
Presolve time: 0.00s
Presolved: 8 rows, 12 columns, 24 nonzeros
Variable types: 0 continuous, 12 integer (12 binary)

Root relaxation: objective 3.400000e+01, 6 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0      34.0000000   34.00000  0.00%

In [5]:
n = 6
random.seed(10)
cities = [f"C{i+1}" for i in range(n)]
coords = {c: (random.randint(0, 20), random.randint(0, 20)) for c in cities}
distances = {}
for i, j in itertools.permutations(cities, 2):
  xi, yi = coords[i]
  xj, yj = coords[j]
  distances[(i, j)] = math.dist((xi, yi), (xj, yj))
m = gp.Model("TSP_large")
x = m.addVars(distances.keys(), vtype=GRB.BINARY, name="x")
m.setObjective(gp.quicksum(distances[i, j] * x[i, j] for (i, j) in distances), GRB.MINIMIZE)
for k in cities:
  m.addConstr(gp.quicksum(x[i, j] for (i, j) in distances if i == k) == 1)
  m.addConstr(gp.quicksum(x[i, j] for (i, j) in distances if j == k) == 1)
m.optimize()

print("City coordinates:")
for c, (xpos, ypos) in coords.items():
  print(f"  {c}: ({xpos}, {ypos})")

print("\nTour edges:")
edges = [(i, j) for (i, j) in distances if x[i, j].X > 0.5]
print(edges)
print("Total cost:", m.objVal)

Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 12 rows, 30 columns and 60 nonzeros
Model fingerprint: 0x62b6071f
Variable types: 0 continuous, 30 integer (30 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 70.2039969
Presolve time: 0.00s
Presolved: 12 rows, 30 columns, 60 nonzeros
Variable types: 0 continuous, 30 integer (30 binary)

Root relaxation: objective 2.780404e+01, 13 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0      27.8040394   27.80404  0.